In [1]:
!pip install dmol-book

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.5 MB 9.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 352 kB 48.3 MB/s 
     |████████████████████████████████| 29.5 MB 1.7 MB/s 
     |████████████████████████████████| 4.7 MB 48.3 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 128 kB 51.2 MB/s 
     |████████████████████████████████| 117 kB 46.0 MB/s 
     |████████████████████████████████| 250 kB 47.8 MB/s 
     |████████████████████████████████| 3.2 MB 48.1 MB/s 
     |████████████████████████████████| 47 kB 5.6 MB/s 
     |████████████████████████████████| 154 kB 48.5 MB/s 
     |████████████████████████████████| 57 kB 5.4 MB/s 
     |████████████████████████

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp
import jax
import dmol
#import seaborn as sns
import rdkit, rdkit.Chem, rdkit.Chem.Draw
import tensorflow as tf
import urllib
from collections import Counter


from google.colab import drive 
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [13]:
dataset = pd.read_csv("gdrive/MyDrive/dataset/CSD_EES_DB.csv")
print(len(dataset))
dataset.head()


48182


,ID,doi,formula,NAts,SMILES,HOMO,LUMO,E(S1),f(S1),E(S2),f(S2),E(S3),f(S3),E(T1),E(T2),E(T3)
0,AACFAZ10,10.1107/S0108270183005156,C26 H22 Cl2 N2 O6,36.0,COC1=C(C(OC1=O)c1ccccc1Cl)C(C)=NN=C(C)C1=C(OC)...,-7.548438,-1.912144,3.1382,0.0027,3.8297,1.1448,4.5310,0.0000,2.2270,2.8977,3.0885
1,ABACIR,10.1107/S160053681103861X,C17 H21 N3 O4 S2,26.0,O=N(=O)c1ccccc1NCC1=CSC(=C1)S(=O)(=O)NC1CCCCC1,-7.675787,-1.725202,3.6720,0.0982,3.7456,0.0888,4.1316,0.0016,2.3255,2.8794,3.2735
2,ABADEO,10.1107/S1600536811039158,C30 H28 N6 O2,38.0,CN1N(c2ccccc2)C(=O)C(=C1C)N=Cc1ccccc1C=NC1=C(C...,-6.603659,-0.762191,3.9027,0.7900,4.0170,0.2765,4.1450,0.0455,2.6892,3.0039,3.5964
3,ABADUE,10.1107/S1600536811039110,C17 H17 N1 O4,22.0,COc1cc(OC)c(C=CC(=O)c2ccccn2)c(OC)c1,-6.790057,-0.969814,3.3127,0.0001,3.9203,0.7088,4.6824,0.0265,2.5848,2.9294,3.9060
4,ABAGAN,10.1107/S1600536811038177,C26 H17 O1 P1,28.0,O=P1(c2ccccc2)c2ccc3ccccc3c2c2c3ccccc3ccc12,-7.067613,-1.419890,3.6197,0.1814,4.2742,0.0073,4.4154,0.0007,2.5192,3.2095,3.8388


In [14]:
## Removing broken SMILES entries

badboys = []
for i in reversed(range(len(dataset))): ## go backwards through list so indexes don't get messed up during the iteration
  try:
    m = rdkit.Chem.MolFromSmiles(dataset.SMILES[i])
    m = rdkit.Chem.AddHs(m)
    #print(i)
  except: 
    badboys.append(i)
print("there are " + str(len(badboys)) + " evil values:")
print( badboys)

for i in range(len(badboys)):
  n = badboys[i]
  dataset = dataset.drop([dataset.index[n]])  ## remove bad values from dataset
dataset = dataset.reset_index(drop=True)
print("New dataset length = " + str(len(dataset)))

there are 457 evil values:
[47815, 47814, 47813, 47812, 47631, 47589, 47588, 47443, 47442, 47441, 47439, 47438, 47437, 47436, 47434, 47433, 47432, 47366, 47365, 47364, 47347, 47345, 47301, 47300, 47291, 47263, 47249, 47248, 47212, 47210, 47209, 47208, 47195, 47006, 47000, 46999, 46998, 46996, 46995, 46994, 46993, 46992, 46991, 46985, 46984, 46983, 46980, 46926, 46925, 46899, 46896, 46698, 46697, 46103, 45913, 45842, 45813, 45334, 45267, 45084, 45083, 45082, 45015, 45013, 44175, 44137, 44120, 44076, 44060, 44059, 44058, 43980, 43979, 43976, 43948, 43677, 43661, 43654, 43460, 43459, 43458, 42246, 42219, 42121, 42120, 42104, 42070, 42069, 42068, 41852, 41851, 41850, 41849, 41788, 41786, 41785, 41643, 41642, 41614, 41612, 41538, 41536, 41535, 41312, 40406, 40401, 40393, 40392, 40391, 40381, 40322, 40321, 40320, 40318, 40316, 40315, 40314, 40313, 40312, 40231, 40080, 40079, 40040, 40039, 40038, 39835, 39612, 39485, 39484, 37692, 37663, 37657, 37626, 37197, 37168, 37167, 37126, 37093, 37092,

In [15]:
## Certain atoms only occur very rarely in the dataset. To simplify the data for the neural network, any molecule containing the least common atoms should be removed.

allatoms = []

for i in reversed(range(len(dataset))):
  m = rdkit.Chem.MolFromSmiles(dataset.SMILES[i])
  for atom in m.GetAtoms():
    allatoms.append(atom.GetSymbol())

print(len(allatoms))

Counter(allatoms)


1411271


Counter({'C': 1131983,
         'O': 114054,
         'S': 26019,
         'N': 99671,
         'P': 2749,
         'Br': 4050,
         'Cl': 6381,
         'F': 19547,
         'B': 2126,
         'Si': 2338,
         'I': 812,
         'Se': 1433,
         'As': 108})

In [16]:
Imolecules = []
Semolecules = []
Asmolecules = []
normalmolecules = []
for i in reversed(range(len(dataset))):
  m = rdkit.Chem.MolFromSmiles(dataset.SMILES[i])
  for atom in m.GetAtoms():
    if atom.GetSymbol() == "I":
      Imolecules.append(dataset.index[i])
    elif atom.GetSymbol() == "Se":
      Semolecules.append(dataset.index[i])
    elif atom.GetSymbol() == "As":
      Asmolecules.append(dataset.index[i])
    else:
      normalmolecules.append(dataset.index[i])

allbadmolecules = Asmolecules + Semolecules + Imolecules
allbadmolecules.sort(reverse=True)

print("instances of I: " + str(len(Imolecules)))
print("instances of Se: " + str(len(Semolecules)))
print("instances of As: " + str(len(Asmolecules)))
print("Atoms that are none of these: " + str(len(normalmolecules)))

Molecules containing I: 812
Molecules containing Se: 1433
Molecules containing As: 108
Molecules containing none of these: 1408918


In [20]:
print(allbadmolecules)  ## SOME INDEXES APPEAR MORE THAN ONCE IN THIS LIST DUE TO HAVING MORE THAN ONE OF THE UNWANTED ATOMS. TO AVOID PROBLEMS LATER, THE DUPLICATES NEED TO BE REMOVED
print(len(allbadmolecules))

nodupe = []
[nodupe.append(x) for x in allbadmolecules if x not in nodupe]
print(nodupe) ## these are the indexes of all molecules in the dataset that contain uncommon atoms.
print(len(nodupe))

for i in range(len(nodupe)):
  n = nodupe[i]
  dataset = dataset.drop([dataset.index[n]])  ## remove bad values from dataset
dataset = dataset.reset_index(drop=True)
print("New dataset length = " + str(len(dataset)))

[47607, 47476, 47476, 47378, 47355, 47355, 47355, 47355, 47333, 47322, 47322, 47300, 47300, 47300, 47300, 47298, 47298, 47208, 47126, 47126, 47119, 47119, 47100, 47100, 47087, 47087, 47078, 47078, 47078, 47078, 47077, 47077, 47077, 47077, 47069, 47049, 47039, 47039, 46923, 46923, 46908, 46908, 46908, 46908, 46907, 46907, 46884, 46812, 46808, 46808, 46807, 46807, 46791, 46791, 46780, 46780, 46749, 46748, 46722, 46720, 46720, 46720, 46720, 46719, 46719, 46718, 46718, 46718, 46718, 46717, 46717, 46716, 46716, 46680, 46672, 46672, 46661, 46661, 46661, 46643, 46633, 46633, 46633, 46633, 46615, 46605, 46605, 46578, 46527, 46475, 46475, 46447, 46447, 46447, 46447, 46328, 46327, 46326, 46325, 46292, 46252, 46252, 46189, 46160, 46040, 46040, 46005, 45969, 45969, 45879, 45848, 45671, 45671, 45657, 45650, 45649, 45582, 45581, 45564, 45485, 45462, 45439, 45412, 45412, 45248, 45165, 45162, 45135, 45134, 45131, 45130, 45129, 45128, 45005, 44876, 44776, 44775, 44740, 44708, 44663, 44638, 44638, 44597